Below is a demo on how to use this brew recipe builder.

The first thing to do is find the ingredients you want to use in the recipe. You can search the tables in the database within the notebook using the code below. As a note, the tables you most likely want to search are: fermentable, hop, yeast and style.

In [1]:
from brew_builder import BrewBuild, search_db, menu_select
import numpy as np
import sqlite3

# connect to the database

con = sqlite3.connect('database/default_db.sqlite')

In [2]:
# search some table using connection to database
# this will display dataframe with search results
# you can type in the search box to change results
menu_select('fermentable', con)

interactive(children=(Text(value='Dry', description='fermentable:', placeholder='Type something'), Output()), …

If what you are looking for doesn't exist in the database, you can add new rows using the function below.

In [3]:
from brew_builder import add_row_table

# database table name to add row to
table_name = 'yeast'

# columns to add values for
columns = ['name', 'form', 'laboratory', 'attenuation', 'min_temperature', 'max_temperature', 'notes']

# values of these columns
values = ['Safale S-33', 'Dry', 'Safale', 75.0, 15, 20,
          ' fruity driven strain that gives a high mouthfeel and body to the beer. ']

add_row_table(con, table_name, columns, values)

When searching for ingredients, it will be important to make notes of the id of the ingredient. You will need this to build the recipe in the next step.

Once you find all of your ingredients, you can now build a recipe! This is done by initilzing the object below:

In [3]:
# set up the grain bill
# the format of each row is [id, amount (lbs), type (0=mash, 1=extract)]
grain_bill = np.array([[9, 0.25, 0],
                       [122, 1, 0],
                       [93, 1, 1],
                       [90, 1, 1],
                       [13, 1.5, 0],
                       [24, 0.5, 0]])

# set up the hop bill
# the format of each row is [id, amount (oz), boil time (min)]
hop_bill = np.array([[62, 1, 45],
                     [21, 0.5, 20]])

# yeast id
yeast = 3

# target volume in gallons
target_volume = 2.5

# boil volume in gallons
boil_volume = 1.75

# mash temp in F
mash_temp = 165

# boil time in min
boil_time = 45

# estimated mash efficiency
mash_efficiency = 48

# the style id
style = 64

# the mash volume in gallons
mash_volume = 1.5

build = BrewBuild(grain_bill, hop_bill, yeast, target_volume,
                               boil_volume, mash_temp, con, boil_time=boil_time,
                               mash_efficiency=mash_efficiency, style=style, mash_volume=mash_volume)

Once the object is setup, below are examples of some of the basic calculations you can do

In [4]:
build.calc_OG()

1.056

In [5]:
build.calc_FG()

1.017

In [6]:
build.calc_ABV(build.calc_OG(), build.calc_FG())

5.12

In [7]:
build.calc_color()

42.0

In [8]:
build.calc_IBU()

27.2

You can also check out the dataframes with your various ingredients (and edit them as well to effect changes in the recipie)

In [9]:
build.df_grain_bill

,id,name,ftype,amount,yield,color,add_after_boil,origin,supplier,notes,...,max_in_batch,recommend_mash,is_mashed,ibu_gal_per_lb,display_unit,display_scale,deleted,display,folder,inventory_id
0,9,Black (Patent) Malt,Grain,0.0,55.0,500.0,false,US,,"The darkest of all malts, use sparingly to add...",...,10.0,false,false,0.0,-1,-1,0,1,,9
1,13,Briess - 6 Row Brewers Malt,Grain,0.0,78.0,1.8,false,US,Briess,DP 180. Base malt for all beer styles. Contrib...,...,100.0,true,true,0.0,-1,-1,0,1,,13
2,24,Briess - Chocolate Malt,Grain,0.0,60.0,350.0,false,US,Briess,Use in all beer styles for color adjustment. U...,...,10.0,false,false,0.0,-1,-1,0,1,,24
3,90,Dry Extract (DME) - Amber,Dry Extract,0.0,95.0,13.0,false,US,,,...,100.0,false,false,0.0,-1,-1,0,1,,90
4,93,Dry Extract (DME) - Light,Dry Extract,0.0,95.0,8.0,false,US,,,...,100.0,false,false,0.0,-1,-1,0,1,,93
5,122,"Oats, Flaked",Grain,0.0,80.0,1.0,false,US,,Oats will improve mouth feel and add a creamy ...,...,30.0,true,true,0.0,-1,-1,0,1,,122


You can also then just build the recipe, which does all of the calculations and creates your recipe in a csv file.

In [10]:
build.build_recipe('Oatmeal_Stout_Test.csv')

You may not be happy with the initial recipe though. So, after you have built the recipe once, you can create an interactive spreadsheet in the notebook which allows you to edit the recipe and see the effects of the changes in real time. Currently, you cant change any of the ingredients in the cells, just their values. The way to think about this sheet, it you can really change anything in a yellow cell and everything in red will be recalculated.

In [11]:
sheet = build.interactive_sheet()

In [12]:
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Summary'), Cell(co…

After making these changes in the spreadsheet above, you can actually then rewrite your recipe based on the changes you have made! This is done below.

In [14]:
build.update_recipe_from_sheet(sheet, 'Oatmeal_Stout_Edited_Test.csv')

Also, you probably don't want to lose your editing from above (in python), so you can aslo pickle your BrewBuild object to open and play with again later!

In [15]:
# pickle the build
build.pickle_build('Oatmeal_Stout_test.pkl')

In [16]:
from brew_builder import build_from_pickle

# open the build again

build_pkl = build_from_pickle('Oatmeal_Stout_test.pkl', con)

We can check this build now and confirm it is the same as what we edited and saved!

In [18]:
build_pkl.build_recipe('Oatmeal_Stout_Edited_Test.csv')

In [19]:
sheet = build_pkl.interactive_sheet()

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Summary'), Cell(co…

Once done, it is best to then close out the connection to the database and get brewing!

In [20]:
con.close()